# Data Engineering Zoomcamp
### Maria Fisher 

## Module 1 Homework

## Docker & SQL

## Question 1. Knowing docker tags

Run the command to get information on Docker 

```docker --help```

Now run the command to get help on the "docker build" command:

```docker build --help```

Do the same for "docker run".

Which tag has the following text? - *Automatically remove the container when it exits* 

- `--delete`
- `--rc`
- `--rmc`
- `--rm` (X)


## Question 2. Understanding docker first run 

Run docker with the python:3.9 image in an interactive mode and the entrypoint of bash.
Now check the python modules that are installed ( use ```pip list``` ). 

What is version of the package *wheel* ?

- 0.42.0 (x)
- 1.0.0
- 23.0.1
- 58.1.0


# Prepare Postgres

Run Postgres and load data as shown in the videos
We'll use the green taxi trips from September 2019:

```wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-09.csv.gz```

You will also need the dataset with zones:

```wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv```

Download this data and put it into Postgres (with jupyter notebooks or with a pipeline)



## Question 3. Count records 

How many taxi trips were totally made on September 18th 2019?

Tip: started and finished on 2019-09-18. 

Remember that `lpep_pickup_datetime` and `lpep_dropoff_datetime` columns are in the format timestamp (date and hour+min+sec) and not in date.

- 15767
- 15612 (x)
- 15859
- 89009

In [ ]:
import pandas as pd
import psycopg2

In [ ]:
import pandas as pd
import psycopg2

# establish a connection to the database
conn = psycopg2.connect(
    host="localhost",
    dbname="ny_taxi",
    user="root",
    password="root"
)

# create a cursor object to execute SQL queries
cur = conn.cursor()

# execute the SQL query to count the number of taxi trips on September 18th, 2019
cur.execute("""
    SELECT COUNT(*) FROM green_taxi_data
    WHERE 
        CAST(lpep_pickup_datetime AS DATE) = '2019-09-18' AND 
        CAST(lpep_dropoff_datetime AS DATE) = '2019-09-18'
""")

# fetch the result of the query
result = cur.fetchone()[0]

# print the number of taxi trips
print(f"Total taxi trips on September 18th, 2019: {result}")

# close the cursor and connection
cur.close()
conn.close()

## Question 4. Largest trip for each day

Which was the pick up day with the largest trip distance
Use the pick up time for your calculations.

- 2019-09-18
- 2019-09-16
- 2019-09-26
- 2019-09-21

In [ ]:
import pandas as pd
import psycopg2

# Connect to the Postgres database
conn = psycopg2.connect(
    host="localhost",
    dbname="ny_taxi",
    user="root",
    password="root"
)
# Create a cursor object
cur = conn.cursor()

# Execute the SQL query to get the pick up day with the largest trip distance
cur.execute("""
    SELECT 
        lpep_pickup_datetime,
        trip_distance
    FROM 
        green_taxi_data
    WHERE 
        DATE(lpep_pickup_datetime) IN ('2019-09-18', '2019-09-16', '2019-09-26', '2019-09-21')

    ORDER BY trip_distance DESC;
    
""")

# Fetch the result
result = cur.fetchone()

# Print the pick up day with the largest trip distance
print("Pick up day with the largest trip distance:", result[0])

# Close the cursor and connection
cur.close()
conn.close()

## Join the tables 


In [ ]:
import psycopg2
import pandas as pd

# Connect to the Postgres database
conn = psycopg2.connect(
    host="localhost",
    dbname="ny_taxi",
    user="root",
    password="root"
)


# Create the SQL query
query = ('''
   SELECT
        CAST(lpep_pickup_datetime AS DATE),
        CAST(lpep_dropoff_datetime AS DATE),
        "total_amount",
        "tip_amount",
        zpu."Borough" AS "zpu_local",
        zpu."Zone"  AS "zpu_zone",
        zdo."Borough" AS "zdo_local",
        zdo."Zone"  AS "zdo_zone"
    FROM
        green_taxi_data t,
        zones zpu,
        zones zdo
    WHERE
        t."PULocationID" = zpu."LocationID" AND
        t."DOLocationID" = zdo."LocationID"
    ORDER BY
        "lpep_pickup_datetime" DESC;
    
''')

# Read the results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()




In [ ]:
df

In [ ]:
df.to_csv('ny_taxi_.csv', index=False)

## Question 5. Three biggest pick up Boroughs

Consider lpep_pickup_datetime in '2019-09-18' and ignoring Borough has Unknown

Which were the 3 pick up Boroughs that had a sum of total_amount superior to 50000?
 
- "Brooklyn" "Manhattan" "Queens" (x)
- "Bronx" "Brooklyn" "Manhattan"
- "Bronx" "Manhattan" "Queens" 
- "Brooklyn" "Queens" "Staten Island"


In [ ]:
import pandas as pd
df = pd.read_csv("ny_taxi_.csv")
df

In [ ]:
lpep_pickup_datetime, lpep_dropoff_datetime,PULocationID, DOLocationID, tip_amount, 

In [ ]:
set0 = df.loc[df["lpep_pickup_datetime"] == "2019-09-18"]
set0

In [ ]:
set1 = set0[set0.zpu_local.isin(["Brooklyn", "Manhattan", "Queens","Staten Island","Bronx" ])]
set1

In [ ]:
set1 = set1.groupby('zpu_local')['total_amount'].sum()
set1 


## Question 6. Largest tip

For the passengers picked up in September 2019 in the zone name Astoria which was the drop off zone that had the largest tip?
We want the name of the zone, not the id.

Note: it's not a typo, it's `tip` , not `trip`

- Central Park
- Jamaica
- JFK Airport
- Long Island City/Queens Plaza (x)

In [ ]:
import psycopg2
import pandas as pd

# Connect to the Postgres database
conn = psycopg2.connect(
    host="localhost",
    dbname="ny_taxi",
    user="root",
    password="root"
)


# Create the SQL query
query = ('''
 SELECT 
    CAST(lpep_pickup_datetime AS DATE) ,
    CAST(lpep_dropoff_datetime AS DATE),
    total_amount,
    tip_amount,
    zpu."Borough" AS "zpu_local",
    zpu."Zone"  AS "zpu_zone",
    zdo."Borough" AS "zdo_local",
    zdo."Zone"  AS "zdo_zone",
    (SELECT SUM(tip_amount) FROM green_taxi_data 
    WHERE CAST(lpep_pickup_datetime AS DATE) >= '2019-09-01' AND 
    CAST(lpep_pickup_datetime AS DATE) < '2019-10-01' AND 
    zpu."Zone" = 'Astoria') AS total_tips_for_astoria

FROM
    green_taxi_data t
JOIN
    zones zpu ON t."PULocationID" = zpu."LocationID"
JOIN
    zones zdo ON t."DOLocationID" = zdo."LocationID"
ORDER BY
    tip_amount DESC; 

''')

# Read the results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()





/tmp/ipykernel_21396/1559433941.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [ ]:
df

In [ ]:
set0 = df.loc[(df["lpep_pickup_datetime"] >= "2019-09-01") & (df['lpep_pickup_datetime'] <= '2019-09-31')]
set0


In [ ]:
setb = set0[set0.zpu_zone.isin(["Astoria"])]
setb

In [ ]:
seta = setb.zdo_zone.unique()


In [ ]:
setb = setb.groupby('zdo_zone')['tip_amount', ].sum()
setb

In [ ]:

df = setb.sort_values(by='tip_amount', ascending=False)

In [ ]:
df

Question 7. Terraform

In this section homework we'll prepare the environment by creating resources in GCP with Terraform.

In your VM on GCP/Laptop/GitHub Codespace install Terraform. 
Copy the files from the course repo
[here](https://github.com/DataTalksClub/data-engineering-zoomcamp/tree/main/01-docker-terraform/1_terraform_gcp/terraform) to your VM/Laptop/GitHub Codespace.

Modify the files as necessary to create a GCP Bucket and Big Query Dataset.


## Question 7. Creating Resources

After updating the main.tf and variable.tf files run:

```
terraform apply
```

Paste the output of this command into the homework submission form.

![terraform bucket](./terraform/terraform_apply.png)

![terraform bucket](./terraform/terraform1_bucket.png)

![terraform bucket](./terraform/terraform2_bigquery.png)